<a href="https://colab.research.google.com/github/vincent2o1/CRX-Lite/blob/main/CRX_Lite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Focusing agent
A function that process piece by piece information about the input source and send it to the ring tree.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#FOCUSING AGENT

import cv2
import numpy as np
import matplotlib.pyplot as plt
import zlib
import base64
import os
import json
from datetime import datetime
from skimage.metrics import structural_similarity as ssim
import numpy as np
# Global storage for agent movements and spot IDs
agent_movements = {}
spot_ids = {}

def image_to_pixels(image_path):
    """
    Convert an image to grayscale pixel values and resize it to 64x64.
    """
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if image is None:
        raise ValueError("Image not found or invalid format.")

    image = cv2.resize(image, (64, 64), interpolation=cv2.INTER_AREA)
    return image

def visualize_agent_movements(image, movements, title="Agent Movements"):
    resized_image = cv2.resize(image, (64, 64), interpolation=cv2.INTER_AREA)
    plt.figure(figsize=(6, 6))
    plt.imshow(resized_image, cmap='gray')

    y_vals, x_vals = zip(*movements)
    plt.plot(x_vals, y_vals, marker='o', color='red', linestyle='-')

    plt.title(title)
    plt.show()

def visualize_spots(image, movements, focus_size=(16, 16)):
    focus_h, focus_w = focus_size
    num_spots = len(movements)
    cols = min(4, num_spots)
    rows = (num_spots + cols - 1) // cols  # Calculate rows to fit all spots

    plt.figure(figsize=(cols * 4, rows * 4))

    for i, (y, x) in enumerate(movements):
        spot = image[y:y+focus_h, x:x+focus_w]
        plt.subplot(rows, cols, i + 1)
        plt.imshow(spot, cmap='gray')
        plt.axis('off')
        plt.title(f"Spot {i+1}")

    plt.suptitle("Focused Spots")
    plt.tight_layout()
    plt.show()

def list_to_reversible_id(pixel_list):
    array = np.array(pixel_list, dtype=np.uint8)
    compressed = zlib.compress(array.tobytes())
    encoded = base64.urlsafe_b64encode(compressed).decode()
    return encoded

def id_to_list(image_id, original_length):
    compressed = base64.urlsafe_b64decode(image_id)
    image_bytes = zlib.decompress(compressed)
    pixel_list = np.frombuffer(image_bytes, dtype=np.uint8).tolist()
    return pixel_list[:original_length]

def calculate_variance(region):
    return np.var(region)

def is_unique_spot(y, x, image, focus_h, focus_w):
    spot = image[y:y+focus_h, x:x+focus_w].flatten().tolist()
    spot_id = list_to_reversible_id(spot)
    if spot_id in spot_ids:
        return False
    spot_ids[spot_id] = (y, x)
   # save_spot_ids_to_drive(spot_id)
    return True

def generate_importance_map(image, threshold_var=50, threshold_edge=30):
    """
    Generate an importance map across the entire image based on variance and edge detection.

    Args:
        image: Input grayscale image
        threshold_var: Variance threshold for important regions
        threshold_edge: Edge detection threshold

    Returns:
        Binary importance map where 1 indicates important regions
    """
    # Initialize importance map
    importance_map = np.zeros_like(image, dtype=np.uint8)

    # Detect edges using Canny edge detection
    edges = cv2.Canny(image, threshold_edge, threshold_edge * 3)

    # Calculate local variance across the entire image using a sliding window
    variance_map = np.zeros_like(image, dtype=np.float32)
    kernel_size = 5  # Size for local variance calculation

    # Pad image for boundary handling
    padded_image = cv2.copyMakeBorder(image, kernel_size//2, kernel_size//2,
                                      kernel_size//2, kernel_size//2,
                                      cv2.BORDER_REFLECT)

    # Calculate variance for each pixel position
    for y in range(image.shape[0]):
        for x in range(image.shape[1]):
            window = padded_image[y:y+kernel_size, x:x+kernel_size]
            variance_map[y, x] = np.var(window)

    # Mark pixels as important if they have high variance or are part of edges
    importance_map = np.logical_or(variance_map > threshold_var, edges > 0).astype(np.uint8)

    # Optional: Apply morphological operations to clean up the map
    kernel = np.ones((3, 3), np.uint8)
    importance_map = cv2.dilate(importance_map, kernel, iterations=1)

    return importance_map

def focusing_agent(image, image1, importance_map, dual_process=False,
                  request_input=None, training=False, focus_size=(16, 16),
                  similarity_threshold=0.8, testing=False):
    """
    Focusing agent that returns stored spots that match with the input image.
    Uses SSIM matching, rotation, and limited zoom in/out.
    """
    height, width = image.shape
    focus_h, focus_w = focus_size
    global spot_ids
    spot_ids = retrieve_all_spot_ids_from_drive_all()

    # Testing mode - extract all familiar spots with selective matching
    if testing:
        focused_regions = []   # Will store the matched stored spots
        focused_regions1 = []  # Will store the matched stored spots (secondary image)
        movements = []         # Will store the positions where matches were found
        matched_positions = set()  # Track positions we've already checked to avoid duplicates

        # Convert all stored spot IDs to images once for efficiency
        stored_spots = {}
        for stored_id in spot_ids.keys():
            try:
                stored_spot_pixels = id_to_list(stored_id, focus_h * focus_w)
                stored_spots[stored_id] = np.array(stored_spot_pixels, dtype=np.uint8).reshape(focus_h, focus_w)
            except Exception as e:
                print(f"Error loading stored spot {stored_id}: {e}")
                continue

        # For each position in the image
        step_size = 1  # Reduced from 2 to ensure we don't miss matches
        for y in range(0, height - focus_h + 1, step_size):
            for x in range(0, width - focus_w + 1, step_size):
                # Create position key for tracking
                pos_key = (y, x)
                if pos_key in matched_positions:
                    continue

                # Skip if not an important region
                region_importance = importance_map[y:y+focus_h, x:x+focus_w]
                if not np.any(region_importance > 0):
                    continue

                # Extract the current spot
                current_spot = image[y:y+focus_h, x:x+focus_w]
                best_match = None
                best_similarity = 0

                # Check against all stored spots
                for stored_id, stored_spot in stored_spots.items():
                    try:
                        # 1. Basic SSIM matching
                        basic_similarity = ssim(current_spot, stored_spot, data_range=255)
                        similarity = basic_similarity

                        if similarity <= similarity_threshold:
                            # 2. Rotation matching - try different angles
                            for angle in [90, 180, 270]:
                                rotated_spot = np.rot90(current_spot, k=angle//90)
                                rot_similarity = ssim(rotated_spot, stored_spot, data_range=255)
                                if rot_similarity > similarity:
                                    similarity = rot_similarity

                            # 3. Limited zoom in/out if still no match
                            if similarity <= similarity_threshold:
                                for scale_factor in [0.95, 1.05]:
                                    # Scale the current spot
                                    scaled_size = int(focus_h * scale_factor)
                                    if scaled_size <= 0 or scaled_size >= height:
                                        continue

                                    scaled_spot = cv2.resize(current_spot, (scaled_size, scaled_size))

                                    # Crop or pad to match original size
                                    if scale_factor > 1:  # Larger than original - crop center
                                        start = (scaled_size - focus_h) // 2
                                        scaled_spot = scaled_spot[start:start+focus_h, start:start+focus_h]
                                    else:  # Smaller than original - pad with zeros
                                        pad_size = (focus_h - scaled_size) // 2
                                        scaled_spot = np.pad(scaled_spot, ((pad_size, pad_size), (pad_size, pad_size)),
                                                           'constant', constant_values=0)

                                        # Ensure exact dimensions
                                        scaled_spot = cv2.resize(scaled_spot, (focus_h, focus_w))

                                    scale_similarity = ssim(scaled_spot, stored_spot, data_range=255)
                                    if scale_similarity > similarity:
                                        similarity = scale_similarity

                        # Track the best match for this position
                        if similarity > similarity_threshold and similarity > best_similarity:
                            best_similarity = similarity
                            best_match = stored_id

                    except Exception as e:
                        print(f"Error in spot matching: {e}")
                        continue

                # If we found a match at this position
                if best_match:
                    # Add the stored spot to the results
                    matched_spot = stored_spots[best_match]
                    focused_regions.append(matched_spot)
                    focused_regions1.append(matched_spot)
                    movements.append((y, x))

                    # Mark this position as matched
                    matched_positions.add(pos_key)

                    # Also mark nearby positions to avoid redundant matches (optional)
                    buffer = 2  # Adjust based on how close matches should be allowed
                    for by in range(max(0, y-buffer), min(height-focus_h+1, y+buffer+1)):
                        for bx in range(max(0, x-buffer), min(width-focus_w+1, x+buffer+1)):
                            matched_positions.add((by, bx))

        if not movements:
            print("No familiar spots found in the image")
            return None

        print(f"Found {len(movements)} familiar spots in the image")
        visualize_agent_movements(image, movements)
        visualize_spots(image, movements)

        # Return the stored spots that matched with the input image
        return focused_regions, focused_regions1

    # Storage for already processed spots to ensure uniqueness
    unique_spots = []
    unique_spot_hashes = set()

    if training:
        focused_regions = []
        movements = []
        checked_positions = set()

        # Find all possible spot positions
        possible_positions = []
        for y in range(0, height - focus_h + 1):
            for x in range(0, width - focus_w + 1):
                # Consider this position only if it contains important pixels
                region_importance = importance_map[y:y+focus_h, x:x+focus_w]
                if np.any(region_importance > 0):
                    possible_positions.append((y, x))

        # Sort positions by importance (number of important pixels in region)
        sorted_positions = sorted(
            possible_positions,
            key=lambda pos: np.sum(importance_map[pos[0]:pos[0]+focus_h, pos[1]:pos[1]+focus_w]),
            reverse=True
        )

        # Process positions in order of importance
        for y, x in sorted_positions:
            if (y, x) in checked_positions:
                continue

            region = image[y:y+focus_h, x:x+focus_w]

            # Skip if variance is too low
            if calculate_variance(region) <= 50:
                checked_positions.add((y, x))
                continue

            # Check if this spot is unique compared to all previously found spots
            spot_is_unique = True
            region_flat = region.flatten()
            spot_hash = hash(region_flat.tobytes())

            # First check simple hash for quick elimination
            if spot_hash in unique_spot_hashes:
                spot_is_unique = False
            else:
                # Also check using structural similarity to catch visually similar spots
                for prev_region, _ in unique_spots:
                    similarity = ssim(region, prev_region, data_range=255)
                    if similarity > similarity_threshold:
                        spot_is_unique = False
                        break

            if spot_is_unique:
                unique_spot_hashes.add(spot_hash)
                unique_spots.append((region, (y, x)))
                movements.append((y, x))

                # Create spot ID for the global dictionary
                spot_id = list_to_reversible_id(region.flatten().tolist())
                spot_ids[spot_id] = (y, x)

                # Mark surrounding positions as checked to avoid nearly duplicate spots
                for dy in range(-focus_h//2, focus_h//2 + 1, focus_h//4):
                    for dx in range(-focus_w//2, focus_w//2 + 1, focus_w//4):
                        ny, nx = y + dy, x + dx
                        if (0 <= ny <= height - focus_h and
                            0 <= nx <= width - focus_w):
                            checked_positions.add((ny, nx))
            else:
                checked_positions.add((y, x))

        if not movements:
            print("Input fully processed. No new unique important portions found.")
            return None

        visualize_agent_movements(image, movements)
        visualize_spots(image, movements)

        focused_regions = [image[y:y+focus_h, x:x+focus_w] for y, x in movements]
        focused_regions1 = [image1[y:y+focus_h, x:x+focus_w] for y, x in movements]

        return focused_regions, focused_regions1
    # Existing logic for dual_process remains unchanged
    if dual_process and request_input is not None:
        match_found = False
        requested_position = None

       # print("0")
        request_input = id_to_list(request_input, 1024)
        # Ensure request_input is a NumPy array
        if isinstance(request_input, list):
            request_input = np.array(request_input)
        # Ensure request_input is in a 2D format (assuming it's meant to be 16x16)
        side_length = int(np.sqrt(len(request_input)))

        if side_length * side_length != len(request_input):
            print("Error: request_input cannot form a square image.")
            return "Error: Invalid input dimensions."

        request_input_2d = request_input.reshape((side_length, side_length))

        # Resize to the target dimensions
        request_input_resized = cv2.resize(request_input_2d, (focus_w, focus_h))
       # print(request_input_resized)

        #print(request_input_resized)
        for y in range(height - focus_h + 1):
            #print("1")
            for x in range(width - focus_w + 1):
                #print("2")
                region = image[y:y+focus_h, x:x+focus_w]
                similarity = ssim(region.astype(np.float32), request_input_resized.astype(np.float32), data_range=255)
                if similarity > 0.40:
                    print("Match found in image source")
                    requested_position = (y, x)
                    match_found = True
                    break
            if match_found:
                break
       # print(f"match found - {match_found}")
        if not match_found:
            return match_found

        request_input_id = list_to_reversible_id(request_input.flatten().tolist())
       # print("2")
       # print(f"spot - {spot_ids}")
        # Convert spot_ids to a list of tuples (key, value)
        spot_id_items = list(spot_ids.items())

        if request_input_id in spot_ids:

            print("Matching ID found")
            current_position = spot_ids[request_input_id]
            print(f"Current position: {current_position}")

            current_index = list(spot_ids.values()).index(current_position)
            #make sure this next id is present in input source.
            if current_index + 1 < len(spot_ids):
                #print("3")
                next_position = list(spot_ids.values())[current_index + 1]
                next_spot_id_key, next_position_ = spot_id_items[current_index + 1]
                print("Connected spot found")
               # spot_image = image[next_position[0]:next_position[0]+focus_h,
                                    #next_position[1]:next_position[1]+focus_w]
                #plt.figure()
                #plt.imshow(spot_image, cmap='gray')
               # plt.title("Connected Spot Visualization")
                #plt.show()
                return (next_spot_id_key, match_found)

def save_spot_ids_to_drive(spot_ids):
    """Save spot IDs with a counter for uniqueness."""
    drive_folder = '/content/drive/MyDrive/ring tree/ids'
    os.makedirs(drive_folder, exist_ok=True)  # Ensure directory exists

    # Find existing files and determine the next counter
    existing_files = [f for f in os.listdir(drive_folder) if f.startswith('spot_ids_') and f.endswith('.json')]
    next_counter = len(existing_files) + 1

    # Create a filename using a counter
    file_path = os.path.join(drive_folder, f'spot_ids_{next_counter:04d}.json')

    try:
        with open(file_path, 'w') as f:
            json.dump(spot_ids, f, indent=4)
        print(f"Spot IDs successfully saved to {file_path}")
        return file_path
    except Exception as e:
        print(f"Error saving spot IDs: {e}")
        return None

def save_spot_ids_to_drive_test(spot_ids):
    """Save spot IDs with a counter for uniqueness."""
    drive_folder = '/content/drive/MyDrive/ring tree/ids_test'
    os.makedirs(drive_folder, exist_ok=True)  # Ensure directory exists

    # Find existing files and determine the next counter
    existing_files = [f for f in os.listdir(drive_folder) if f.startswith('spot_ids_') and f.endswith('.json')]
    next_counter = len(existing_files) + 1

    # Create a filename using a counter
    file_path = os.path.join(drive_folder, f'spot_ids_{next_counter:04d}.json')

    try:
        with open(file_path, 'w') as f:
            json.dump(spot_ids, f, indent=4)
        print(f"Spot IDs successfully saved to {file_path}")
        return file_path
    except Exception as e:
        print(f"Error saving spot IDs: {e}")
        return None

def store_spot_ids_to_drive_all(spot_ids):
    """Store the given spot IDs in a JSON file in Google Drive within one unified dictionary."""
    drive_folder = '/content/drive/MyDrive/ring tree/ids_all'
    os.makedirs(drive_folder, exist_ok=True)  # Ensure directory exists

    file_path = os.path.join(drive_folder, 'spot_ids.json')

    try:
        # Load existing data or initialize an empty dictionary
        if os.path.exists(file_path):
            with open(file_path, 'r') as f:
                existing_data = json.load(f)
        else:
            existing_data = {}

        # Append or update data without overwriting keys
        for key, value in spot_ids.items():
            if key in existing_data:
                print(f"Key {key} already exists. Skipping.")
            else:
                existing_data[key] = value

        with open(file_path, 'w') as f:
            json.dump(existing_data, f, indent=4)
        print(f"Spot IDs successfully stored to {file_path}")
        return file_path
    except Exception as e:
        print(f"Error storing spot IDs: {e}")
        return None

def retrieve_all_spot_ids_from_drive_all():
    """Retrieve all stored spot IDs from Google Drive."""
    drive_folder = '/content/drive/MyDrive/ring tree/ids_all'
    file_path = os.path.join(drive_folder, 'spot_ids.json')

    try:
        if not os.path.exists(file_path):
            print("No spot IDs found.")
            return {}

        with open(file_path, 'r') as f:
            spot_ids = json.load(f)
            print(f"Successfully retrieved {len(spot_ids)} spot ID entries.")
            return spot_ids
    except Exception as e:
        print(f"Error retrieving spot IDs: {e}")
        return {}

def compare_focus(focus_list1, focus_list2):
    """
    Compare two lists of focus values and print whether they are the same or different.
    """
    if len(focus_list1) != len(focus_list2):
        print("❗ The focus lists have different lengths. They are not the same.")
        return

    for i, (f1, f2) in enumerate(zip(focus_list1, focus_list2)):
        if np.array_equal(f1, f2):
            print(f"✅ Focus {i + 1} is the same.")
        else:
            print(f"❗ Focus {i + 1} is different.")
           # print(f"  Focus 1: {f1}")
           # print(f"  Focus 2: {f2}")

def visualize_processed_pixels(focused_regions):
    """
    Visualizes the processed portions from both input images side by side.
    """
    num_regions = len(focused_regions)
    if num_regions == 0:
        print("No processed regions to visualize.")
        return

    plt.figure(figsize=(10, 5))

    for i in range(num_regions):
        # Reshape to 2D (Assuming 32x32 images)
        reshaped_image = np.array(focused_regions[i], dtype=np.uint8).reshape(16, 16)

        plt.subplot(1, num_regions, i + 1)
        plt.imshow(reshaped_image, cmap='gray')
        plt.title(f"Region {i+1}")
        plt.axis('off')

    plt.suptitle("Processed Image Regions")
    plt.tight_layout()
    plt.show()

def visualize_importance_map(image, importance_map):
    """
    Visualize the importance map overlaid on the original image.
    """
    plt.figure(figsize=(12, 5))

    plt.subplot(1, 3, 1)
    plt.title("Original Image")
    plt.imshow(image, cmap='gray')
    plt.axis('off')

    plt.subplot(1, 3, 2)
    plt.title("Importance Map")
    plt.imshow(importance_map, cmap='hot')
    plt.axis('off')

    plt.subplot(1, 3, 3)
    plt.title("Important Regions Overlay")
    # Create a 3-channel image for color overlay
    overlay = np.stack([
        np.where(importance_map > 0, 255, image),  # Red channel - highlight important areas
        image,  # Green channel
        image   # Blue channel
    ], axis=-1)

    plt.imshow(overlay)
    plt.axis('off')

    plt.tight_layout()
    plt.show()

image_path = 'input image :'
image_path1 = 'input image :' # testing image to compare the training and testing
training = True
testing = False
focus = []
focus2_ = []

if training == True:
    image = image_to_pixels(image_path)
    image1 = image_to_pixels(image_path1)
    focus, focus2 = focusing_agent(image, image1, training = True)
    compare_focus(focus, focus2)

    for focus_ in focus2: # for testing purpose
      focus__ = list_to_reversible_id(focus_)
      focus2_.append(focus__)
      save_spot_ids_to_drive_test(focus__)

    focus_ids = []
    for focus1 in focus:
        focus_id = list_to_reversible_id(focus1)
        focus_ids.append(focus_id)
        save_spot_ids_to_drive(focus_id)

    print("id comparison")
    compare_focus(focus_ids, focus2_)
    print("test visualization")
    visualize_processed_pixels(focus2)

    store_spot_ids_to_drive_all(spot_ids)

if testing == True:
    image = image_to_pixels(image_path)
    image1 = image_to_pixels(image_path1)
    importance_map = generate_importance_map(image)
    visualize_importance_map(image, importance_map)

    result = focusing_agent(image, image1, importance_map, testing=True)

    if result:
        focus1, focus2 = result
        focus2_ = []  # Reset the list before appending

        for focus_ in focus1: # For testing purpose
            focus__ = list_to_reversible_id(focus_.flatten().tolist())
            focus2_.append(focus__)
            save_spot_ids_to_drive_test(focus__)

        print("Test visualization")
        visualize_processed_pixels(focus2)
    else:
        print("No familiar spots to process")


# Ring Tree
the successful patterns/ID gets stored in a comman directory where ring tree can access these stored files and organize them in a ring tree structure, and assists in **storage**, **retrieval**, **removal of stored unwanted information**


STORAGE - On training, the Ring tree function takes a successful patterns and organize them in a ring tree structure


---


RETRIEVAL - on testing, the model takes the input signals and retrieves the stored information


---


REMOVAL OF STORED INFORMATION - the ring tree function inherently works on cleaning the stored unwanted process. it does this process automatically.

**DETAILS ABOUT THE RING TREE**

*General rules*

*   if branched, the dual process should get more info to confirm the path to travel
*  searcher should get more input (atleast 5) to confirm the ring tree specificity to the input.
*   initially give 2 outputs for each confirmation of nodes with the input, after confrimation happens for 3 consecutive nodes, increase the output number to 4.


---




*Specific rules*
1. Storage of ring tree - should be sequentially one by one. output should be given by travelling down the tree


---


2. Branching - only if present in main root.

*   should check for atleast 3 rings similarity to the 3 consecutive inputs to confirm that main root ring to create as branch.
*   if atleast 5 rings are same then directy create connection within the main root, no need of branch.




---


3. Insertion of new ring - only if not present in main root + not found when cross-ring connection


---


4. Cross ring connection - only if donar ring tree is activated by input source. each activated ring from donar ring tree should be taken for testing similarity with the receiver ring tree activated leftalone ring to find a good match and then proceed with the receiver ring tree processing. leave the donar tree


---


5. Dual process - should actively communicate with focusing agent to gather additional input


---


**TRAINING** - Feeds all the piece of information into the model.


---


**ORGANIZING** - Organizes the incoming already familiar information within the ring tree netork


---


**TESTING** - A function that enables comprehensive traversal of interconnected 'ring trees' based on incoming input and external confirmation via focusing agent.

In [ ]:
#RING TREE

import json
import time
import os
import base64
import zlib
import numpy as np
import matplotlib.pyplot as plt

output2 = []
importance_map = generate_importance_map(image)

def ring_tree(data=None, training=False, organizing=False, testing=False, continuation=False, previous_result=None, **kwargs):
    # Load existing state when the function is first called
    if not hasattr(ring_tree, 'surface'):
        # Try to load existing state from drive
        load_ring_tree_from_drive()

        # If loading fails or no previous state, initialize
        if not hasattr(ring_tree, 'surface'):
            ring_tree.surface = {}
            ring_tree.surface_rankings = {"R'": {}, "R''": {}, "R'''": {}}
            ring_tree.access_counts = {}
            ring_tree.windows = {}
            ring_tree.branches = {}
            ring_tree.branch_connections = {}
            ring_tree.tree_counter = 1
            ring_tree.branch_counter = 1
            ring_tree.current_tree = None

        ring_tree.min_trees_for_ranking = 10
        ring_tree.activation_thresholds = {"detach": 10}

    def compare_values(val1, val2):
        if isinstance(val1, tuple) and isinstance(val2, tuple):
            if len(val1) != len(val2):
                return False
            return all(abs(a - b) < 0.001 for a, b in zip(val1, val2))
        return val1 == val2

    def dict_match(dict1, dict2):

        # Directly compare if both are not dictionaries
        if not isinstance(dict1, dict) and not isinstance(dict2, dict):
            return dict1 == dict2

        # Ensure both are dictionaries for further checks
        if not isinstance(dict1, dict) or not isinstance(dict2, dict):
            return False

        # Check if keys match
        if set(dict1.keys()) != set(dict2.keys()):
            return False

        # Check values using compare_values
        for key in dict1:
            if key not in dict2 or not compare_values(dict1[key], dict2[key]):
                return False
        return True

    def update_rankings():
        """Update rankings of trees based on access patterns"""
        # Only use ranking system if we have enough trees
        if len(ring_tree.surface) < ring_tree.min_trees_for_ranking:
            return "Not enough trees for ranking system"

        # Calculate total access for each tree
        tree_access = {}
        for tree_name, counts in ring_tree.access_counts.items():
            if tree_name in ring_tree.surface:  # Only consider existing trees
                tree_access[tree_name] = sum(counts)

        # Sort trees by total access
        sorted_trees = sorted(tree_access.items(), key=lambda x: x[1], reverse=True)

        # Reset rankings
        for rank in ring_tree.surface_rankings:
            ring_tree.surface_rankings[rank] = []

        # Distribute trees into ranking buckets
        total_trees = len(sorted_trees)
        r_prime_count = max(1, total_trees // 3)  # At least 1 tree in R'
        r_double_prime_count = max(1, total_trees // 3)  # At least 1 tree in R''

        # Assign R' (top third or at least 1)
        for i in range(min(r_prime_count, len(sorted_trees))):
            ring_tree.surface_rankings["R'"].append(sorted_trees[i][0])

        # Assign R'' (middle third or at least 1)
        for i in range(r_prime_count, min(r_prime_count + r_double_prime_count, len(sorted_trees))):
            ring_tree.surface_rankings["R''"].append(sorted_trees[i][0])

        # Assign R''' (bottom third or remaining)
        for i in range(r_prime_count + r_double_prime_count, len(sorted_trees)):
            ring_tree.surface_rankings["R'''"].append(sorted_trees[i][0])

        r1 = len(ring_tree.surface_rankings["R'"])
        r2 = len(ring_tree.surface_rankings["R''"])
        r3 = len(ring_tree.surface_rankings["R'''"])
        return f"Updated rankings: R': {r1} trees, R'': {r2} trees, R''': {r3} trees"

    def find_matching_prefix(input_data, min_prefix=2):
        """Find the longest matching prefix between input_data and any existing tree"""
        best_match = (None, -1, 0)  # (tree_name, start_idx, match_length)
        # If we have enough trees to use ranking system, prioritize by rank
        if len(ring_tree.surface) > 0:
            for rank in ["R'", "R''", "R'''"]:
                for tree_name in ring_tree.surface_rankings.get(rank, []):
                    tree_data = ring_tree.surface.get(tree_name, [])
                    # Check if the beginning of input_data matches beginning of tree_data
                    match_length = 0

                    for j in range(len(tree_data)):
                       # print(f"Comparing input_data[{j}] with all tree data")
                        for i in range(2):

                            if dict_match(input_data[i], tree_data[j]):
                                match_length += 1
                            else:
                                pass

                    if match_length >= min_prefix:
                        best_match = (tree_name, 0, match_length)
                        # Early return for efficiency if we find a match in high-ranked trees
                        if rank == "R'":
                            return best_match

    def find_return_point(branch_data, main_tree):
        """Find where the branch can return to the main tree"""
        tree_data = ring_tree.surface[main_tree]

        # Check the last node in branch data
        last_node = branch_data[-1]
        for idx, node in enumerate(tree_data):
            if dict_match(last_node, node):
                # Found a matching node - return to tree from this point
                return idx, idx + 1

        return None, None

    def create_branch(tree_name, initial_branch_data, common_prefix_length):
        branch_name = f"branch_{ring_tree.branch_counter}"
        ring_tree.branch_counter += 1

        # Initialize accumulated data with the initial branch data
        accumulated_data = initial_branch_data.copy()
        continue_gathering = True

        # Keep gathering data until we find something that doesn't match the main tree
        while continue_gathering:
            # Call focusing_agent to get additional data
            result = focusing_agent(image, image1, dual_process=True, request_input=accumulated_data[-1])

            if isinstance(result, tuple) and len(result) == 2:
                additional_data, boolean_flag = result
                print(f"Additional data: {additional_data}, Boolean Flag: {boolean_flag}")
            else:
                boolean_flag = result
                print(f"Boolean Flag: {boolean_flag}")
                additional_data = None

            if boolean_flag is None or additional_data is None:
                print("No more additional data. Ending data gathering.")
                break

            # Check if additional data exists in main tree
            data_in_main_tree = False
           # for node in additional_data:
            for tree_data_node in ring_tree.surface.get(tree_name, []):
                if dict_match(additional_data, tree_data_node):
                    data_in_main_tree = True
                    break

            # Add the additional_data as a whole item, not character by character
            if isinstance(additional_data, str):
                accumulated_data.append(additional_data)
            elif isinstance(additional_data, list):
                accumulated_data.extend(additional_data)
            else:
                print(f"Warning: Unexpected type for additional_data: {type(additional_data)}")
                # Try to add it anyway
                accumulated_data.append(additional_data)

            if data_in_main_tree:
                # Data matches with main tree, continue gathering
                print(f"Data found in main tree. Continuing to gather more data.")
            else:
                # Found data that doesn't match main tree, stop gathering
                print(f"Data not found in main tree. Stopping data gathering.")
                continue_gathering = False

        # Now create the branch with all accumulated data
        # Only now find return path to tree
        return_from_idx, return_to_idx = find_return_point(accumulated_data, tree_name)
        # Store branch info
        ring_tree.branches[branch_name] = {
            "reference_tree": tree_name,
            "reference_length": common_prefix_length,
            "unique_data": accumulated_data,
            "returns_to_tree": return_to_idx is not None,
            "return_from_idx": return_from_idx,
            "return_to_idx": return_to_idx
        }

        # Store connection information
        branch_off_point = common_prefix_length - 1
        ring_tree.branch_connections[branch_name] = {
            "parent_tree": tree_name,
            "branch_point_index": branch_off_point,
            "returns_to_tree": return_to_idx is not None,
            "return_from_branch_idx": len(accumulated_data) - 1 if return_to_idx is not None else None,
            "return_to_tree_idx": return_to_idx
        }

        message = f"Created branch {branch_name} from {tree_name}\n"
        message += f"Common prefix: {accumulated_data}\n"
        message += f"Branch-specific data: {accumulated_data[common_prefix_length:]}\n"

        if return_to_idx is not None:
            message += f"Branch returns to tree at index {return_to_idx}\n"

        return message

    def update_tree_with_new_nodes(tree_name, match_length, data):
        """Update the specified tree with new nodes from data, inserting at the appropriate position"""
        # Get the existing tree data
        tree_data = ring_tree.surface[tree_name]

        # Determine which nodes need to be added (those beyond match_length)
        nodes_to_add = data[match_length:]

        # Calculate insertion position (right after the matching prefix)
        insertion_position = match_length

        # Insert the new nodes at the specified position
        for i, node in enumerate(nodes_to_add):
            tree_data.insert(insertion_position + i, node)
            # Also update the access counts for the new nodes
            ring_tree.access_counts[tree_name].insert(insertion_position + i, 0)

        # Update the indices for any window pointers or other references
        if ring_tree.windows[tree_name] >= insertion_position:
            ring_tree.windows[tree_name] += len(nodes_to_add)

        # Update connection indices that might be affected by the insertion
        for conn_name, conn_data in ring_tree.branch_connections.items():
            if conn_data["parent_tree"] == tree_name and conn_data["branch_point_index"] >= insertion_position:
                conn_data["branch_point_index"] += len(nodes_to_add)

            # Update similarity nodes indices if needed
            for i, sim_node in enumerate(conn_data.get("similarity_nodes", [])):
                idx, sim_tree, sim_idx = sim_node
                if sim_tree == tree_name and sim_idx >= insertion_position:
                    conn_data["similarity_nodes"][i] = (idx, sim_tree, sim_idx + len(nodes_to_add))

        # Return information about the update
        return f"Inserted {len(nodes_to_add)} new nodes at position {insertion_position} in {tree_name}"

    def update_access_counts(tree_name, node_index):
        """Update access counts for a node and check if window should be moved"""
        if tree_name in ring_tree.access_counts:
            # Ensure access_counts array is large enough
            if len(ring_tree.access_counts[tree_name]) <= node_index:
                # Extend the array to accommodate the node index
                extension = [0] * (node_index - len(ring_tree.access_counts[tree_name]) + 1)
                ring_tree.access_counts[tree_name].extend(extension)

            # Now we can safely increment the access count
            ring_tree.access_counts[tree_name][node_index] += 1

            # Update window to most frequently accessed node
            max_access = max(ring_tree.access_counts[tree_name])
            max_index = ring_tree.access_counts[tree_name].index(max_access)
            old_window = ring_tree.windows.get(tree_name, 0)
            ring_tree.windows[tree_name] = max_index

            # Check if access count exceeds threshold for detaching
            if ring_tree.access_counts[tree_name][max_index] >= ring_tree.activation_thresholds["detach"]:
                return detach_window_to_new_tree(tree_name, max_index)

            return f"Updated window for {tree_name} from {old_window} to {max_index}, access count now: {ring_tree.access_counts[tree_name][node_index]}"
        return "No access count updated - tree not found"

    def detach_window_to_new_tree(tree_name, window_index):
        """
        Detach a window of continuous activated nodes and all following nodes
        from the original tree and create a new single tree with them.
        """
        if tree_name not in ring_tree.surface:
            return f"Tree {tree_name} not found"

        tree_data = ring_tree.surface[tree_name]
        access_counts = ring_tree.access_counts[tree_name]
        tree_size = len(tree_data)

        # Find a continuous sequence of activated nodes (3-5 nodes)
        # Start with the most activated node
        window_start = window_index
        window_end = window_index + 1

        # Look for activated nodes before the window_index (up to 2 positions)
        for i in range(1, 3):
            prev_idx = (window_index - i) % tree_size
            # Only include if it's consecutive (physically adjacent) and has been activated
            if prev_idx == (window_index - i) and access_counts[prev_idx] > 0:
                window_start = prev_idx
            else:
                break  # Stop if we hit a non-activated node or non-consecutive position

        # Look for activated nodes after the window_index (up to 2 positions)
        for i in range(1, 3):
            next_idx = (window_index + i) % tree_size
            # Only include if it's consecutive and has been activated
            if next_idx == (window_index + i) and access_counts[next_idx] > 0:
                window_end = next_idx + 1
            else:
                break  # Stop if we hit a non-activated node or non-consecutive position

        # Check if we have at least 3 continuous nodes in our window
        if window_end - window_start < 3:
            return f"Not enough continuous activated nodes around {window_index} in {tree_name} to form a window"

        # Create a new tree from the window area plus all following nodes
        new_tree_name = f"tree_{ring_tree.tree_counter}_from_{tree_name}"
        ring_tree.tree_counter += 1

        # Copy nodes from the window start to the end of the tree
        # This includes both the activated window and all following nodes
        new_tree_data = tree_data[window_start:]

        # Store as a new tree
        ring_tree.surface[new_tree_name] = new_tree_data

        # Initialize access counts for new tree
        ring_tree.access_counts[new_tree_name] = [0] * len(new_tree_data)
        ring_tree.windows[new_tree_name] = 0

        # Update the original tree to only contain nodes before the window
        ring_tree.surface[tree_name] = tree_data[:window_start]
        ring_tree.access_counts[tree_name] = access_counts[:window_start]

        # Add new tree to highest rank if ranking system is active
        if len(ring_tree.surface) >= ring_tree.min_trees_for_ranking:
            if new_tree_name not in ring_tree.surface_rankings["R'"]:
                ring_tree.surface_rankings["R'"].append(new_tree_name)

        update_rankings()

        result_message = (f"Detached window of {window_end - window_start} continuous nodes "
                        f"plus {len(new_tree_data) - (window_end - window_start)} following nodes "
                        f"to new tree {new_tree_name}. "
                        f"Original tree {tree_name} now has {len(ring_tree.surface[tree_name])} nodes")

        return result_message

    def consecutive_match_(tree_data, data, index):
        # Check for 3 consecutive matches
        for i in range(3):
            compare_index = (index + i) % len(tree_data)
            data_index = i % len(data)
            if not dict_match(tree_data[compare_index], data[data_index]):
                return False
        return True

    def searching_via_common_information(data):
        # Call the focusing agent twice to check activation
        donar_tree = None
        for _ in range(2):
            focus, focus2 = focusing_agent(image, image1, training=True)
            focus_ids = [list_to_reversible_id(f) for f in focus]

            # Search for matches in high-ranked ring trees
            for rank in ["R'", "R''", "R'''"]:
                for tree_name in ring_tree.surface_rankings.get(rank, []):
                    tree_data = ring_tree.surface.get(tree_name, [])

                    # Check for 3 consecutive matches using focus_ids
                    for index in range(len(tree_data) - 2):
                        if consecutive_match_(tree_data, focus_ids, index):
                            donar_tree = tree_name
                            break
                    if donar_tree:
                        break
                if donar_tree:
                    break

            if donar_tree:
                break  # Stop checking if a donor tree is found

        if not donar_tree:
            return False  # No donor tree activated, try another method

        # Find receiver tree using existing matching logic
        receiver_tree, start_idx, match_length = find_matching_prefix(data, min_prefix=3)

        if not receiver_tree or receiver_tree == donar_tree:
            return "Receiver tree not found or same as donar tree."

        # Create connections for nodes that exist in other trees
        connections_created = []

        for i in range(match_length, len(data)):
            node = data[i]
            found_in_donar = None
            found_in_receiver = None

            # Search for node in donar tree
            for index, existing_node in enumerate(ring_tree.surface[donar_tree]):
                if dict_match(node, existing_node):
                    found_in_donar = index
                    break

            # Search for node in receiver tree
            for index, existing_node in enumerate(ring_tree.surface[receiver_tree]):
                if dict_match(node, existing_node):
                    found_in_receiver = index
                    break

            if found_in_donar is not None and found_in_receiver is None:
                # Connect from donar to receiver
                our_idx = match_length + (i - match_length)
                connection_name = f"connection_{ring_tree.branch_counter}"
                ring_tree.branch_counter += 1

                ring_tree.branch_connections[connection_name] = {
                    "parent_tree": receiver_tree,
                    "branch_point_index": our_idx,
                    "connected_tree": donar_tree,
                    "connected_index": found_in_donar,
                    "connection_type": 'cross_tree',
                    "similarity_nodes": [(i, donar_tree, found_in_donar)]
                }

                # Create reverse connection
                connection_name2 = f"connection_{ring_tree.branch_counter}"
                ring_tree.branch_counter += 1

                ring_tree.branch_connections[connection_name2] = {
                    "parent_tree": donar_tree,
                    "branch_point_index": found_in_donar,
                    "connected_tree": receiver_tree,
                    "connected_index": our_idx,
                    "connection_type": 'cross_tree',
                    "similarity_nodes": [(i, donar_tree, found_in_donar)]
                }

                connections_created.append((receiver_tree, our_idx, donar_tree, found_in_donar))

        if connections_created:
            connections_str = ", ".join([f"{c[0]}[{c[1]}]↔{c[2]}[{c[3]}]" for c in connections_created])
            return f"Connections established: {connections_str}"
        else:
            return False

    # Training mode: create and populate trees
    if training and data:
        if ring_tree.current_tree is None or len(ring_tree.surface.get(ring_tree.current_tree, [])) >= 20:
            ring_tree.current_tree = f"tree_{ring_tree.tree_counter}"
            ring_tree.tree_counter += 1
            ring_tree.surface[ring_tree.current_tree] = []
            #ring_tree.surface_rankings["R'"][ring_tree.current_tree] = {}
            ring_tree.access_counts[ring_tree.current_tree] = []
            ring_tree.windows[ring_tree.current_tree] = 0

        ring_tree.surface[ring_tree.current_tree].append(data)
        if "R'" not in ring_tree.surface_rankings:
            ring_tree.surface_rankings["R'"] = {}

        if ring_tree.current_tree not in ring_tree.surface_rankings["R'"]:
            ring_tree.surface_rankings["R'"][ring_tree.current_tree] = []

        ring_tree.surface_rankings["R'"][ring_tree.current_tree].append(data)

        ring_tree.access_counts[ring_tree.current_tree].append(0)

        # Check if ranking system should be activated
        if len(ring_tree.surface) == ring_tree.min_trees_for_ranking:
            update_rankings()
            message = f"Stored in {ring_tree.current_tree}: {data} (Item {len(ring_tree.surface[ring_tree.current_tree])} of 5). Ranking system activated!"
        else:
            message = f"Stored in {ring_tree.current_tree}: {data} (Item {len(ring_tree.surface[ring_tree.current_tree])} of 5)"

            # If we already have enough trees, update rankings
            if len(ring_tree.surface) > ring_tree.min_trees_for_ranking:
                update_rankings()

        return message

    # Organizing mode: handle the sequence according to the logic
    if organizing and data:

        # Call focusing_agent to get additional data if needed
        result = focusing_agent(image, image1, dual_process=True, request_input=data)
=
        if isinstance(result, tuple) and len(result) == 2:
            data_2, boolean_flag = result
            print(f"Data 2: {data_2}, Boolean Flag: {boolean_flag}")
        else:
            boolean_flag = result
            print(f"Boolean Flag: {boolean_flag}")
        if boolean_flag is None:
            print("Boolean flag is None. Ending function.")
            return

        # Combine data and data_2 for further processing
        combined_data = [data, data_2]
=
        # Check if combined_data is already in the same order in the ring tree
        def check_existing_order():
            # Check main root
            for tree_name in ring_tree.surface:
                tree_data = ring_tree.surface[tree_name]

                # Check if combined_data appears in sequence in tree_data
                for i in range(len(tree_data) - len(combined_data) + 1):
                    matches = True
                    for j in range(len(combined_data)):
                        if not dict_match(combined_data[j], tree_data[i + j]):
                            matches = False
                            break
                    if matches:
                        print(f"Combined data already exists in the same order in tree: {tree_name}")
                        return True

                # Check branches
                for branch_name, branch_info in ring_tree.branches.items():
                    if branch_info["reference_tree"] == tree_name:
                        # Reconstruct branch data
                        branch_data = (tree_data[:branch_info["reference_length"]] +
                                      branch_info["unique_data"])

                        # Check if combined_data appears in sequence in branch_data
                        for i in range(len(branch_data) - len(combined_data) + 1):
                            matches = True
                            for j in range(len(combined_data)):
                                if not dict_match(combined_data[j], branch_data[i + j]):
                                    matches = False
                                    break
                            if matches:
                                print(f"Combined data already exists in the same order in branch: {branch_name}")
                                return True

            return False

        # If data is already in correct order, end function
        if check_existing_order():
            return "Data is already correctly ordered in the ring tree."

        # Existing organizing logic for creating branch and node insertion
        match_result = find_matching_prefix(combined_data, min_prefix=2)

        if match_result is None:
            print("No matching prefix found.")
            return
        else:
            tree_name, start_idx, match_length = match_result

        if tree_name:
            # In organizing mode:
            should_create_branch = False
            for i in range(0, 2):
                node = combined_data[i]
                # Check if any exact match exists in any tree
                for tree_data2 in ring_tree.surface.get(tree_name, []):
                    if dict_match(node, tree_data2):
                        should_create_branch = True
                        break
                if should_create_branch:
                    break

            if should_create_branch:
                # Create a branch only if we found exact matches of new nodes in existing trees
                message = create_branch(tree_name, combined_data, match_length)
            else:
                # Try searching via common information before directly updating
                common_information = searching_via_common_information(data)
                if not common_information:
                    message = update_tree_with_new_nodes(tree_name, match_length, data)
            return message

        else:
            # No matching pattern found, create a new tree
            new_tree_name = f"tree_{ring_tree.tree_counter}"
            ring_tree.tree_counter += 1
            ring_tree.surface[new_tree_name] = data.copy()
            ring_tree.access_counts[new_tree_name] = [0] * len(data)
            ring_tree.windows[new_tree_name] = 0

            # Ensure ranking dictionary exists
            if "R'''" not in ring_tree.surface_rankings:
                ring_tree.surface_rankings["R'''"] = []

            # Add to lowest rank if ranking is active
            if len(ring_tree.surface) >= ring_tree.min_trees_for_ranking:
                ring_tree.surface_rankings["R'''"].append(new_tree_name)
                update_rankings()

            message = f"Created new tree {new_tree_name} with {len(data)} nodes."
            return message

    # Testing mode: find and retrieve data with confirmation mechanism
    if testing and data:
        result = {"found": None, "message": "Data not found", "connected": [], "tree": None, "awaiting_confirmation": True}

        def consecutive_match(tree_data, data, index):
            # Check for 2 consecutive matches
            for i in range(2):
                compare_index = (index + i) % len(tree_data)
                data_index = i % len(data)
                if not dict_match(tree_data[compare_index], data[data_index]):
                    return False
            return True

        def get_next_search_index(start_index, connected_nodes, total_length):
            # Determine the next search index, ensuring it is after the connected nodes
            last_connected_index = max([start_index] + connected_nodes)
            return (last_connected_index + 1) % total_length

        # Initialize traversal state
        state = {
            "current_tree": None,
            "current_index": 0,
            "visited_nodes": set(),
            "path": [],
            "matched_rings": [],
            "window_updates": []  # Track window updates during traversal
        }

        # First search in high-ranked trees if ranking is active
        potential_matches = []
        if len(ring_tree.surface) > 0:
            for rank in ["R'", "R''", "R'''"]:
                for tree_name in ring_tree.surface_rankings.get(rank, []):
                    tree_data = ring_tree.surface.get(tree_name, [])

                    for index, item in enumerate(tree_data):
                        if dict_match(item, data):
                            # Update access count for the matched node
                            window_update = update_access_counts(tree_name, index)

                            # Get initial connected nodes (just 2)
                            connected = []
                            for offset in range(1, 3):
                                next_index = (index + offset) % len(tree_data)
                                connected.append(tree_data[next_index])

                            # Continue searching from the node after connected nodes
                            next_search_index = get_next_search_index(index, [index + 1, index + 2], len(tree_data))

                            potential_matches.append({
                                "found": item,
                                "message": f"Found in {tree_name} ({rank} rank) at position {index}",
                                "tree": tree_name,
                                "connected": connected,
                                "type": "tree",
                                "index": index,
                                "next_search_index": next_search_index,
                                "window_update": window_update
                            })

        # If we found potential matches, begin traversal with the first one
        if potential_matches:
            # Start with the first match
            match = potential_matches[0]
            state["current_tree"] = match["tree"]
            state["current_index"] = match["index"]
            state["matched_rings"].append(match["found"])
            if "window_update" in match and match["window_update"]:
                state["window_updates"].append(match["window_update"])
            # Begin traversal
            try:
                while True:
                    # Get current tree data
                    current_tree_data = ring_tree.surface.get(state["current_tree"], [])
                    if not current_tree_data:
                        result = {
                            "status": "error",
                            "message": f"Tree {state['current_tree']} is empty",
                            "matched_rings": state["matched_rings"],
                            "path": state["path"],
                            "window_updates": state["window_updates"]
                        }
                        break

                    # Check if the current node matches our search data
                    current_node = current_tree_data[state["current_index"]]
                    output = id_to_list(current_node, 1024)
                    output2.append(output)

                    node_key = (state["current_tree"], state["current_index"])

                    # Avoid infinite loops by tracking visited nodes
                    if node_key in state["visited_nodes"]:
                        result = {
                            "status": "complete",
                            "message": "Traversal complete - returned to previously visited node",
                            "matched_rings": state["matched_rings"],
                            "path": state["path"],
                            "window_updates": state["window_updates"]
                        }
                        break

                    state["visited_nodes"].add(node_key)
                    state["path"].append({"tree": state["current_tree"], "index": state["current_index"], "data": current_node})

                    # Update access count for this node
                    window_update = update_access_counts(state["current_tree"], state["current_index"])
                    if window_update:
                        state["window_updates"].append(window_update)

                    # Check if there's a branch connection at this point
                    branch_connection = None
                    for conn_name, conn_data in ring_tree.branch_connections.items():
                        if (conn_data["parent_tree"] == state["current_tree"] and
                            conn_data["branch_point_index"] == state["current_index"]):
                            branch_connection = {"name": conn_name, "data": conn_data}
                            break

                    if branch_connection:
                        # Process branch connection
                        branch_tree = branch_connection["data"]["connected_tree"]
                        branch_index = branch_connection["data"]["connected_index"]

                        # Get branch node(s)
                        branch_tree_data = ring_tree.surface.get(branch_tree, [])
                        if not branch_tree_data:
                            result = {
                                "status": "error",
                                "message": f"Branch tree {branch_tree} is empty",
                                "matched_rings": state["matched_rings"],
                                "path": state["path"],
                                "window_updates": state["window_updates"]
                            }
                            break

                        # For a single node branch, just verify it
                        branch_node = branch_tree_data[branch_index]
                        focus, focus2 = focusing_agent(image, image1, dual_process=True, requested_input=branch_node)

                        if not focus2:
                            result = {
                                "status": "error",
                                "message": f"Branch node confirmation failed with focusing agent",
                                "matched_rings": state["matched_rings"],
                                "path": state["path"],
                                "window_updates": state["window_updates"]
                            }
                            break

                        # Add confirmed branch node to our path
                        state["visited_nodes"].add((branch_tree, branch_index))
                        state["path"].append({"tree": branch_tree, "index": branch_index, "data": branch_node})
                        state["matched_rings"].append(branch_node)
                        print(f"Confirmed node: {branch_node}")  # Add this line to see the output
                        # Call this when a node is confirmed
                        output = id_to_list(branch_node, 1024)
                        output2.append(output)
                        # Update access count for branch node
                        window_update = update_access_counts(branch_tree, branch_index)
                        if window_update:
                            state["window_updates"].append(window_update)

                        # Now traverse through branch tree
                        state["current_tree"] = branch_tree
                        state["current_index"] = branch_index
                    else:
                        # No branch, traverse to next node in current tree. no confirmation from focusing agent
                        next_index = (state["current_index"] + 1) % len(current_tree_data)
                        next_node = current_tree_data[next_index]

                        # Verify with focusing agent
                        result = focusing_agent(image, image1, dual_process=True, request_input=next_node)

                        if isinstance(result, tuple) and len(result) == 2:
                            data_2, boolean_flag = result
                            print(f"Data 2: {data_2}, Boolean Flag: {boolean_flag}")
                        else:
                            boolean_flag = result
                            print(f"Boolean Flag: {boolean_flag}")

                        if boolean_flag is None:
                            result = {
                                "status": "error",
                                "message": f"Focusing agent did not confirm next node in sequence",
                                "error_location": {"tree": state["current_tree"], "index": next_index},
                                "matched_rings": state["matched_rings"],
                                "path": state["path"],
                                "window_updates": state["window_updates"]
                            }
                            break

                        # Add confirmed node and update state
                        state["current_index"] = next_index
                        state["matched_rings"].append(next_node)
                        print(f"Confirmed node branch: {next_node}")  # Add this line to see the output
                        print("main root connection node")
                        output = id_to_list(next_node, 1024)
                        output2.append(output)

                        # Update access count for this node
                        window_update = update_access_counts(state["current_tree"], next_index)
                        if window_update:
                            state["window_updates"].append(window_update)

                    if len(state["matched_rings"]) > 1:
                        # Determine how many nodes to output next based on consecutive confirmations
                        confirmation_count = len(state["matched_rings"])
                        if confirmation_count >= 6:
                            nodes_to_output = 4  # Output 4 nodes after 4+ confirmations
                        elif confirmation_count >= 4:
                            nodes_to_output = 2  # Output 2 nodes after 2-3 confirmations
                        else:
                            nodes_to_output = 1  # Default to 1 node (shouldn't reach here due to condition)

                        # Cap at 6 nodes maximum
                        nodes_to_output = min(nodes_to_output, 6)

                        # Get the next node to confirm with focusing agent
                        next_index = (state["current_index"] + 1) % len(current_tree_data)
                        next_node = current_tree_data[next_index]

                        result = focusing_agent(image, image1, importance_map, dual_process=True, request_input=next_node)

                        if isinstance(result, tuple) and len(result) == 2:
                            data_2, boolean_flag = result
                            print(f"Data 2: {data_2}, Boolean Flag: {boolean_flag}")
                        else:
                            boolean_flag = result
                            print(f"Boolean Flag: {boolean_flag}")

                        if boolean_flag is None:
                            result = {
                                "status": "error",
                                "message": f"Focusing agent could not confirm next node in traversal",
                                "error_location": {"tree": state["current_tree"], "index": next_index},
                                "matched_rings": state["matched_rings"],
                                "path": state["path"],
                                "window_updates": state["window_updates"]
                            }
                            break

                        # Check if focus output matches any node in the current tree
                        match_found = False
                        for i, item in enumerate(current_tree_data):
                            if dict_match(item, data_2):
                                # This is the matched node
                                state["current_index"] = i
                                state["matched_rings"].append(item)

                                # Output the confirmed matched node
                                print("Confirmed matched node after initial")
                                output = id_to_list(item, 1024)
                                output2.append(output)

                                # Output the next nodes_to_output (1, 2, or 4) nodes attached to this matched node
                                for offset in range(1, nodes_to_output + 1):
                                    next_attached_index = (i + offset) % len(current_tree_data)
                                    next_attached_node = current_tree_data[next_attached_index]
                                    print(f"Node {offset} after confirmed node")
                                    output = id_to_list(next_attached_node, 1024)
                                    output2.append(output)

                                # Continue traversal with the node after the last one shown
                                next_node_index = (i + nodes_to_output + 1) % len(current_tree_data)
                                state["current_index"] = next_node_index

                                # Update access count for the matched node
                                window_update = update_access_counts(state["current_tree"], i)
                                if window_update:
                                    state["window_updates"].append(window_update)

                                match_found = True
                                break

                        if not match_found:
                            result = {
                                "status": "error",
                                "message": f"No matching node found for focus output in any tree",
                                "matched_rings": state["matched_rings"],
                                "path": state["path"],
                                "window_updates": state["window_updates"]
                            }
                            break

            except Exception as e:
                result = {
                    "status": "error",
                    "message": f"Unexpected error during traversal: {str(e)}",
                    "matched_rings": state["matched_rings"],
                    "path": state["path"],
                    "window_updates": state["window_updates"]
                }
        else:
            result = {"status": "error", "message": "Data not found", "matched_rings": [], "path": [], "window_updates": []}

        return result

    # Print the current state if no specific operation is requested
    if not any([training, organizing, testing]):
        return {
            "surface": ring_tree.surface,
            "surface_rankings": ring_tree.surface_rankings,
            "access_counts": ring_tree.access_counts,
            "windows": ring_tree.windows,
            "branches": ring_tree.branches,
            "branch_connections": ring_tree.branch_connections,
            "visualization": visualize_structures()
        }

def load_spot_ids_from_drive_with_waiting(wait_interval=5):
    """
    Continuously load and return spot IDs from stored JSON files in counter order.
    Remembers the last processed counter using a file.

    Args:
        wait_interval (int): Time (in seconds) to wait before checking again.

    Yields:
        dict: Loaded spot IDs from each processed file.
    """
    drive_folder = '/content/drive/MyDrive/ring tree/ids'
    counter_file_path = '/content/drive/MyDrive/ring tree/last_processed_counter.txt'

    if not os.path.exists(drive_folder):
        print(f"Directory not found at {drive_folder}")
        return

    # Ensure counter file exists
    if not os.path.exists(counter_file_path):
        with open(counter_file_path, 'w') as f:
            f.write('0')
        print("Initialized last processed counter to 0.")

    # Read the last processed counter
    with open(counter_file_path, 'r') as f:
        last_processed_counter = int(f.read().strip())

    while True:
        # List all JSON files
        json_files = [f for f in os.listdir(drive_folder) if f.startswith('spot_ids_') and f.endswith('.json')]

        if not json_files:
            print("No spot ID files found. Waiting...")
            time.sleep(wait_interval)
            continue

        # Sort files by counter value
        json_files.sort(key=lambda x: int(x.split('_')[2].split('.')[0]))

        # Process only new files
        for file_name in json_files:
            file_path = os.path.join(drive_folder, file_name)
            try:
                # Extract counter from filename
                current_counter = int(file_name.split('_')[2].split('.')[0])

                if current_counter <= last_processed_counter:
                    continue  # Skip already processed files

                # Load and yield spot IDs
                with open(file_path, 'r') as f:
                    spot_ids = json.load(f)

                print(f"Processing {file_name}...")

                # Update the counter and delete file
                last_processed_counter = current_counter
                with open(counter_file_path, 'w') as f:
                    f.write(str(last_processed_counter))

                #os.remove(file_path)
                print(f"File {file_name} processed and deleted.")

                # Yield the loaded spot IDs for external processing
                yield spot_ids
            except Exception as e:
                print(f"Error processing {file_name}: {e}")

        # Wait for new files if no more to process
        time.sleep(wait_interval)


def load_spot_ids_from_drive_with_waiting_test(wait_interval=5):
    """
    Continuously load and return spot IDs from stored JSON files in counter order.
    Remembers the last processed counter using a file.

    Args:
        wait_interval (int): Time (in seconds) to wait before checking again.

    Yields:
        dict: Loaded spot IDs from each processed file.
    """
    drive_folder = '/content/drive/MyDrive/ring tree/ids_test'
    counter_file_path = '/content/drive/MyDrive/ring tree/last_processed_counter.txt'

    if not os.path.exists(drive_folder):
        print(f"Directory not found at {drive_folder}")
        return

    # Ensure counter file exists
    if not os.path.exists(counter_file_path):
        with open(counter_file_path, 'w') as f:
            f.write('0')
        print("Initialized last processed counter to 0.")

    # Read the last processed counter
    with open(counter_file_path, 'r') as f:
        last_processed_counter = int(f.read().strip())

    while True:
        # List all JSON files
        json_files = [f for f in os.listdir(drive_folder) if f.startswith('spot_ids_') and f.endswith('.json')]

        if not json_files:
            print("No spot ID files found. Waiting...")
            time.sleep(wait_interval)
            continue

        # Sort files by counter value
        json_files.sort(key=lambda x: int(x.split('_')[2].split('.')[0]))

        # Process only new files
        for file_name in json_files:
            file_path = os.path.join(drive_folder, file_name)
            try:
                # Extract counter from filename
                current_counter = int(file_name.split('_')[2].split('.')[0])

                if current_counter <= last_processed_counter:
                    continue  # Skip already processed files

                # Load and yield spot IDs
                with open(file_path, 'r') as f:
                    spot_ids = json.load(f)

                print(f"Processing {file_name}...")

                # Update the counter and delete file
                last_processed_counter = current_counter
                with open(counter_file_path, 'w') as f:
                    f.write(str(last_processed_counter))

                #os.remove(file_path)
                print(f"File {file_name} processed and deleted.")

                # Yield the loaded spot IDs for external processing
                yield spot_ids
            except Exception as e:
                print(f"Error processing {file_name}: {e}")

        # Wait for new files if no more to process
        time.sleep(wait_interval)


def save_ring_tree_to_drive():
    """Save the current ring tree structure to a JSON file in Google Drive"""
    try:
        # Correct path with consistent naming
        drive_folder = '/content/drive/MyDrive/ring_tree/ring'
        os.makedirs(drive_folder, exist_ok=True)

        # Prepare the data to be saved
        ring_tree_data = {
            'surface': ring_tree.surface,
            'surface_rankings': ring_tree.surface_rankings,
            'access_counts': ring_tree.access_counts,
            'windows': ring_tree.windows,
            'branches': ring_tree.branches,
            'branch_connections': ring_tree.branch_connections,
            'tree_counter': ring_tree.tree_counter,
            'branch_counter': ring_tree.branch_counter,
            'current_tree': ring_tree.current_tree
        }

        # Save to a JSON file
        file_path = os.path.join(drive_folder, 'ring_tree_state.json')
        with open(file_path, 'w') as f:
            json.dump(ring_tree_data, f, indent=4)

        print(f"Ring tree state saved to {file_path}")
        return True
    except Exception as e:
        print(f"Error saving ring tree state: {e}")
        return False

def load_ring_tree_from_drive():
    """Load the ring tree structure from a JSON file in Google Drive"""
    try:
        # Correct the path
        drive_folder = '/content/drive/MyDrive/ring_tree/ring'
        file_path = os.path.join(drive_folder, 'ring_tree_state.json')

        # Check if the folder exists
        if not os.path.exists(drive_folder):
            print(f"Directory not found: {drive_folder}")
            return False

        # Check if the file exists
        if not os.path.isfile(file_path):
            print(f"No existing ring tree state found at {file_path}")
            return False

        # Load the JSON file
        with open(file_path, 'r') as f:
            ring_tree_data = json.load(f)

        # Restore the ring tree state
        ring_tree.surface = ring_tree_data.get('surface', {})
        ring_tree.surface_rankings = ring_tree_data.get('surface_rankings', {"R'": {}, "R''": {}, "R'''" : {}})
        ring_tree.access_counts = ring_tree_data.get('access_counts', {})
        ring_tree.windows = ring_tree_data.get('windows', {})
        ring_tree.branches = ring_tree_data.get('branches', {})
        ring_tree.branch_connections = ring_tree_data.get('branch_connections', {})
        ring_tree.tree_counter = ring_tree_data.get('tree_counter', 1)
        ring_tree.branch_counter = ring_tree_data.get('branch_counter', 1)
        ring_tree.current_tree = ring_tree_data.get('current_tree', None)

        print("Ring tree state loaded successfully")
        return True
    except Exception as e:
        print(f"Error loading ring tree state: {e}")
        return False

def id_to_list(image_id, original_length):
    compressed = base64.urlsafe_b64decode(image_id)
    image_bytes = zlib.decompress(compressed)
    pixel_list = np.frombuffer(image_bytes, dtype=np.uint8).tolist()
    return pixel_list[:original_length]

def visualize_processed_pixels(focused_regions, images_per_row=4, max_images_per_figure=32, save_path="focused_spots_test.png"):
    """
    Visualizes processed image regions in a grid format, creating multiple figures if needed.
    Each figure contains at most max_images_per_figure images with images_per_row per row.
    """
    num_regions = len(focused_regions)
    if num_regions == 0:
        print("No processed regions to visualize.")
        return

    # Calculate number of figures needed
    num_figures = (num_regions + max_images_per_figure - 1) // max_images_per_figure

    for fig_num in range(num_figures):
        # Determine start and end indices for this figure
        start_idx = fig_num * max_images_per_figure
        end_idx = min((fig_num + 1) * max_images_per_figure, num_regions)
        current_batch = focused_regions[start_idx:end_idx]

        # Calculate rows needed for this batch
        batch_size = len(current_batch)
        rows = (batch_size + images_per_row - 1) // images_per_row

        # Create figure
        fig, axes = plt.subplots(rows, images_per_row, figsize=(images_per_row * 3, rows * 3))

        # Handle single row case
        if rows == 1:
            axes = np.array([axes])

        # Handle single image case
        if rows == 1 and images_per_row == 1:
            axes = np.array([[axes]])

        # Plot images in this batch
        for i, ax in enumerate(axes.flatten()):
            if i < batch_size:
                reshaped_image = np.array(current_batch[i], dtype=np.uint8).reshape(16, 16)
                ax.imshow(reshaped_image, cmap='gray')
                ax.set_title(f"Spot {start_idx + i + 1}")
                ax.axis('off')
            else:
                ax.axis('off')  # Hide unused subplots

        # Set title and save
        plt.suptitle(f"Focused Spots (Batch {fig_num + 1} of {num_figures})", fontsize=14)
        plt.tight_layout()

        # Create batch-specific filename
        if num_figures > 1:
            base, ext = os.path.splitext(save_path)
            batch_save_path = f"{base}_batch{fig_num + 1}{ext}"
        else:
            batch_save_path = save_path

        # Save and close figure
        plt.savefig(batch_save_path, dpi=300, bbox_inches='tight')
        print(f"Visualization batch {fig_num + 1} saved as {batch_save_path}")
        plt.close()  # Close the figure to free memory

training = False
testing = True
organizing = False

if training == True:
    for inputs in load_spot_ids_from_drive_with_waiting():
        print(inputs)
        print(ring_tree(inputs, training=True))
        save_ring_tree_to_drive()

elif organizing == True:
    for inputs in load_spot_ids_from_drive_with_waiting():
        print(ring_tree(inputs, organizing=True))
        save_ring_tree_to_drive()
else:
    count = 0
    for inputs in load_spot_ids_from_drive_with_waiting_test():
        #print(f"input - {inputs}")
        #print(f"input - {type(inputs)}")
        print(ring_tree(inputs, testing=True))
        count += 1  # Increment counter
        print(f"count - {count}")
        if count >= 10:  # Stop after 10 iterations
            break
    visualize_processed_pixels(output2)